https://www.kaggle.com/datasets/muhammetzahitaydn/hardhat-vest-dataset-v3/

https://www.kaggle.com/datasets/niravnaik/safety-helmet-and-reflective-jacket/data

In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pickle
from sklearn.metrics import classification_report
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline

In [184]:
!pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 153.5 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 599.4 kB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 2.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [4]:
from ultralytics import YOLO

In [43]:

model = YOLO("yolov8n.pt")
model.info()


results = model.train(data="../data.yaml", epochs=1, imgsz=640)


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics YOLOv8.2.32 🚀 Python-3.11.7 torch-2.3.1 CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning /home/mk1MoreBugs/projects/building_hack/datasets/safety-Helmet-


train: New cache created: /home/mk1MoreBugs/projects/building_hack/datasets/safety-Helmet-Reflective-Jacket/labels/train.cache


val: Scanning /home/mk1MoreBugs/projects/building_hack/datasets/safety-Helmet-Re

val: New cache created: /home/mk1MoreBugs/projects/building_hack/datasets/safety-Helmet-Reflective-Jacket/labels/val.cache


Plotting labels to runs/detect/train14/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train14
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G       1.25      1.556       1.28         23        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1575       5292      0.805       0.72      0.811      0.512

1 epochs completed in 1.918 hours.
Optimizer stripped from runs/detect/train14/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train14/weights/best.pt, 6.2MB

Validating runs/detect/train14/weights/best.pt...
Ultralytics YOLOv8.2.32 🚀 Python-3.11.7 torch-2.3.1 CPU (AMD Ryzen 5 5500U with Radeon Graphics)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1575       5292      0.803      0.721      0.811      0.512
         Safety-Helmet       1290       2966      0.884      0.761      0.866       0.56
     Reflective-Jacket       1317       2326      0.723      0.682      0.755      0.463
Speed: 2.9ms preprocess, 176.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train14


In [46]:
model.export(format="onnx")

Ultralytics YOLOv8.2.32 🚀 Python-3.11.7 torch-2.3.1 CPU (AMD Ryzen 5 5500U with Radeon Graphics)

PyTorch: starting from 'runs/detect/train14/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 2.2s, saved as 'runs/detect/train14/weights/best.onnx' (11.7 MB)

Export complete (3.9s)
Results saved to /home/mk1MoreBugs/projects/building_hack/yolov5/runs/detect/train14/weights
Predict:         yolo predict task=detect model=runs/detect/train14/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train14/weights/best.onnx imgsz=640 data=../data.yaml  
Visualize:       https://netron.app


'runs/detect/train14/weights/best.onnx'

In [5]:

model = YOLO("yolov5/runs/detect/train14/weights/best.pt")
model.info()


Model summary: 225 layers, 3011238 parameters, 0 gradients, 8.2 GFLOPs


(225, 3011238, 0, 8.1952256)

In [8]:
# Run inference with the YOLOv8n model
results = model("./datasets/safety-Helmet-Reflective-Jacket/images/test/helmet_jacket_07364.jpg")


for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


image 1/1 /home/mk1MoreBugs/projects/building_hack/datasets/safety-Helmet-Reflective-Jacket/images/test/helmet_jacket_07364.jpg: 640x640 2 Safety-Helmets, 3 Reflective-Jackets, 158.6ms
Speed: 5.7ms preprocess, 158.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
def yolo_object_detection(im):
    result = model(im)[0]

    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
  #      result.show()  # display to screen
   #     result.save(filename="result.jpg")  # save to disk
    result = result.plot()
    return result

In [24]:
import cv2 as cv
def open_video(file, output_file):
    cap = cv.VideoCapture(file)
    if not cap.isOpened():
         print("Cannot open camera")
         exit()
    out = cv.VideoWriter(output_file,  
                         cv.VideoWriter_fourcc(*"MP4V"),
                         10, (640,480),
                        )

    
    while True:
     # Capture frame-by-frame
        ret, frame = cap.read()
 
     # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame = yolo_object_detection(frame)
        # Display the resulting frame
       # cv.imshow("Video Capture", frame)
        out.write(cv.resize(frame, (640,480)))
        cv.waitKey(1)
     
        if cv.waitKey(1) == ord('q'):
            break
 
    # When everything done, release the capture
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [25]:
open_video("Фундамент под загородный дом __ Цена под ключ!.mp4", 'output2.mp4')
open_video("Фундамент под ключ __ Цена с материалом и работой!.mp4", 'output3.mp4')
open_video("Цена фундамента под дом!.mp4", 'output4.mp4')




0: 384x640 (no detections), 106.0ms
Speed: 2.2ms preprocess, 106.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)



OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0: 384x640 (no detections), 108.0ms
Speed: 2.3ms preprocess, 108.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 97.9ms
Speed: 2.0ms preprocess, 97.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 106.7ms
Speed: 2.2ms preprocess, 106.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 101.4ms
Speed: 2.0ms preprocess, 101.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 105.1ms
Speed: 2.2ms preprocess, 105.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 102.4ms
Speed: 2.0ms preprocess, 102.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 105.7ms
Speed: 2.2ms preprocess, 105.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 102.1ms
Speed: 2.0ms preproce

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0: 384x640 (no detections), 99.6ms
Speed: 2.0ms preprocess, 99.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 95.4ms
Speed: 2.3ms preprocess, 95.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 121.8ms
Speed: 2.2ms preprocess, 121.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 97.3ms
Speed: 2.1ms preprocess, 97.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 116.0ms
Speed: 2.4ms preprocess, 116.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 115.4ms
Speed: 2.6ms preprocess, 115.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 105.5ms
Speed: 3.0ms preprocess, 105.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 104.1ms
Speed: 2.2ms preprocess, 

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0: 384x640 (no detections), 97.3ms
Speed: 1.7ms preprocess, 97.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 113.6ms
Speed: 4.3ms preprocess, 113.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 95.9ms
Speed: 2.8ms preprocess, 95.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 107.9ms
Speed: 2.9ms preprocess, 107.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 96.6ms
Speed: 3.2ms preprocess, 96.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 95.2ms
Speed: 4.7ms preprocess, 95.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 119.1ms
Speed: 3.2ms preprocess, 119.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 98.8ms
Speed: 2.7ms preprocess, 98.